https://mp.weixin.qq.com/s?__biz=MjM5MzY4NzE3MA==&mid=2247484652&idx=1&sn=1ab00e57d902133759044486d6ebcb3f&source=41#wechat_redirect

In [ ]:
# 1.指定GPU编号
os.environ['CUDA_VISIBLE_EVICES'] = '0'

In [ ]:
# 2.查看模型每层的输出详情
from torchsummary import summary
summary(your_model,input_size=(channels,H,W))

In [ ]:
# 3.梯度裁剪
import torch.nn as nn
outputs = model(data)
loss = loss_fn(outputs,target)
optimizer.zero_grad()
loss.backward()
nn.utils.clip_grad_norm_(model.parameters(),max_norm=20,norm_type=2)
optimizer.step()

In [ ]:
# 4.扩展单张图片维度
# 方案1
import cv2
import torch
image = cv2.imread(img_path)
image = torch.tensor(image)
print(image.size())
img = image.view(1,*image.size())
print(img.size())
# 方案2
img = image[np.newaxis,:,:,:]
# 方案3
img = image.unsqueeze(dim=0)

In [2]:
# 5.独热编码
import torch
class_num = 8
batch_size = 4
def one_hot(label):
    label = label.resize_(batch_size,1)
    m_zeros = torch.zeros(batch_size,class_num)
    onehot = m_zeros.scatter(1,label,1)
    return onehot.numpy()
label = torch.LongTensor(batch_size).random_() % class_num
print(label)
print(one_hot(label))

tensor([7, 6, 2, 5])
[[0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]]


In [ ]:
# 6.防止验证模型时爆显存
with torch.no_grad():
    # 使用model进行预测的代码
    pass
torch.cuda.empty_cache()

In [ ]:
# 7.学习率衰减
import torch.optim as optim
from torch.optim import lr_scheduler
# 训练前的初始化
optimizer = optim.Adam(net.parameters(),lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer,10,0.1)
# 训练过程中
for n in n_epoch:
    scheduler.step()
    ...

In [ ]:
# 8.冻结某些层的参数
net = Network() # 获取自定义网络结构
for name,value in net.named_parameters():
    print('name: {0},\t grad: {1}'.format(name, value.requires_grad))
# 假设前几层信息如下：
name: cnn.VGG_16.convolution1_1.weight,	 grad: True
name: cnn.VGG_16.convolution1_1.bias,	 grad: True
name: cnn.VGG_16.convolution1_2.weight,	 grad: True
name: cnn.VGG_16.convolution1_2.bias,	 grad: True
name: cnn.VGG_16.convolution2_1.weight,	 grad: True
name: cnn.VGG_16.convolution2_1.bias,	 grad: True
name: cnn.VGG_16.convolution2_2.weight,	 grad: True
name: cnn.VGG_16.convolution2_2.bias,	 grad: True
# 后面的True表示该层的参数可训练，然后我们定义一个要冻结的层的列表
no_grad = [
    'cnn.VGG_16.convolution1_1.weight',
    'cnn.VGG_16.convolution1_1.bias',
    'cnn.VGG_16.convolution1_2.weight',
    'cnn.VGG_16.convolution1_2.bias'
]
# 冻结方法如下
net = Net.CTPN() # 获取网络结构
for name,value in net.named_parameters():
    if name in no_grad:
        value.requires_grad = False
    else:
        value.requires_grad = True
# 最后在定义优化器时，只对requires_grad为True的层的参数进行更新
optimizer = optim.Adam(filter(lambda p:p.requires_grad,net.parameters()),lr=0.01)

In [ ]:
# 对不同层使用不同学习率
net = Network()  # 获取自定义网络结构
for name, value in net.named_parameters():
    print('name: {}'.format(name))
# 输出：
# name: cnn.VGG_16.convolution1_1.weight
# name: cnn.VGG_16.convolution1_1.bias
# name: cnn.VGG_16.convolution1_2.weight
# name: cnn.VGG_16.convolution1_2.bias
# name: cnn.VGG_16.convolution2_1.weight
# name: cnn.VGG_16.convolution2_1.bias
# name: cnn.VGG_16.convolution2_2.weight
# name: cnn.VGG_16.convolution2_2.bias
# 对convolution1和convlution2设置不同的学习率，首先将它们分开，即放到不同的列表里
conv1_params = []
conv2_params = []
for name,parms in net.named_parameters():
    if "convlution1" in name:
        conv1_params += [parms]
    else:
        conv2_params += [parms]
# 然后在优化器中进行如下操作
optimizer = optim.Adam(
    [
        {"params":conv1_params,"lr":0.01},
        {"params":conv2_params,"lr":0.001},
    ],
    weight_decay = 1e-3,
)